# 1.Trabajando con API's

La intención de este notebook es la de trabajar con API's, en este caso, para la descarga de información para su posterior analisis.

In [1]:
# Importando las librerias
import requests
import pandas as pd
import time

In [4]:
# Parametros para usar el API DE YT
# Amable recordatorio de siempre revisar esta pagina para crear la API}
# https://www.slickremix.com/docs/get-api-key-for-youtube/
# https://console.cloud.google.com/project
# XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

API_KEY = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
CHANNEL_ID = 'UCBNvm8XDKtQhoi6usy4HXAw'

In [5]:
# Prueba de la API
response = requests.get('https://api.github.com').json()

In [6]:
# Si se imprime información vamos por buen camino
response

{'current_user_url': 'https://api.github.com/user',
 'current_user_authorizations_html_url': 'https://github.com/settings/connections/applications{/client_id}',
 'authorizations_url': 'https://api.github.com/authorizations',
 'code_search_url': 'https://api.github.com/search/code?q={query}{&page,per_page,sort,order}',
 'commit_search_url': 'https://api.github.com/search/commits?q={query}{&page,per_page,sort,order}',
 'emails_url': 'https://api.github.com/user/emails',
 'emojis_url': 'https://api.github.com/emojis',
 'events_url': 'https://api.github.com/events',
 'feeds_url': 'https://api.github.com/feeds',
 'followers_url': 'https://api.github.com/user/followers',
 'following_url': 'https://api.github.com/user/following{/target}',
 'gists_url': 'https://api.github.com/gists{/gist_id}',
 'hub_url': 'https://api.github.com/hub',
 'issue_search_url': 'https://api.github.com/search/issues?q={query}{&page,per_page,sort,order}',
 'issues_url': 'https://api.github.com/issues',
 'keys_url': '

## 1.1 Llamada a la API

La estructura de la URL para consultar la data de YT es la siguiente

'https://www.googleapis.com/youtube/v3/search?key='+**API_KEY**+"&channelId="+**CHANNEL_ID**+"&part=snippet,id&order=date&maxResults=10000"

* API_KEY es la key que se genero anteriormente

* CHANNEL_ID del canal objetivo

Para más información, consultar la documentación oficial: https://developers.google.com/youtube/v3/docs/search

In [45]:
# en este caso me interesa buscar la información del canal de Dross
# el CHANNEL_ID es el siguiente UCNYW2vfGrUE6R5mIJYzkRyQ
# en la parte de Results=10000 estamos pidiendo la informacion de los 10000 videos subidos (en donde espero que no exista gente loca que suba tantos videos)

channel_objetive = 'UCNYW2vfGrUE6R5mIJYzkRyQ'
pageToken = ''

url = 'https://www.googleapis.com/youtube/v3/search?key='+API_KEY+'&channelId='+channel_objetive+'&part=snippet,id&order=date&maxResults=10000'+pageToken

In [38]:
# Verificamos si hay respuesta con los datos necesarios
response = requests.get(url=url).json()

In [39]:
# Observamos que los datos se encuentran en forma de diccionario

type(response)

dict

In [40]:
response.keys()

dict_keys(['kind', 'etag', 'nextPageToken', 'regionCode', 'pageInfo', 'items'])

In [52]:
# Se observa el formato JSON del ultimo video subido por Dross

response['items'][0]

{'kind': 'youtube#searchResult',
 'etag': 'M9MajbaBwwFeSDqPHp0WBOo5iU0',
 'id': {'kind': 'youtube#video', 'videoId': 'cFlk_Rd1DGg'},
 'snippet': {'publishedAt': '2023-02-22T02:00:01Z',
  'channelId': 'UCNYW2vfGrUE6R5mIJYzkRyQ',
  'title': 'TOP: Las 7 personas de Twitch más...',
  'description': 'Si el video te gusta, yo valoraría muchísimo que le dieras like, y que comentaras algo. ¡Gracias! ❤ #twitch.',
  'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/cFlk_Rd1DGg/default.jpg',
    'width': 120,
    'height': 90},
   'medium': {'url': 'https://i.ytimg.com/vi/cFlk_Rd1DGg/mqdefault.jpg',
    'width': 320,
    'height': 180},
   'high': {'url': 'https://i.ytimg.com/vi/cFlk_Rd1DGg/hqdefault.jpg',
    'width': 480,
    'height': 360}},
  'channelTitle': 'DrossRotzank',
  'liveBroadcastContent': 'none',
  'publishTime': '2023-02-22T02:00:01Z'}}

## 1.2 Seleccionando informacion

Apartir del JSON generado, es posible seleccionar determinada información, deacuerdo a las necesidades que se requieran

In [53]:
# Obtenemos el Id, titulo y fecha de subida del ultimo video publicado por Dross

video_id = response['items'][0]['id']['videoId']

video_title = response['items'][0]['snippet']['title']
video_title = str(video_title).replace('&amp;', '')

upload_date = response['items'][0]['snippet']['publishedAt']
upload_date = str(upload_date).split('T')[0]

print(f'''ID del video: {video_id}
Titulo del video: {video_title}
Fecha de subida: {upload_date}''')


ID del video: cFlk_Rd1DGg
Titulo del video: TOP: Las 7 personas de Twitch más...
Fecha de subida: 2023-02-22


## 1.3 Recolección de información

In [70]:
for info_video in response['items']:
    # Verificamos que todos los resultados sean videos
    if info_video['id']['kind'] == 'youtube#video':
        video_id = info_video['id']['videoId']

        video_title = info_video['snippet']['title']
        video_title = str(video_title).replace('&amp;', '')

        upload_date = info_video['snippet']['publishedAt']
        upload_date = str(upload_date).split('T')[0]
        
        url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
        response_video_stats = requests.get(url_video_stats).json()
        
        view_count = response_video_stats['items'][0]['statistics']['viewCount']
        like_count = response_video_stats['items'][0]['statistics']['likeCount']
        comment_count = response_video_stats['items'][0]['statistics']['commentCount']
        

print(f'''ID del video: {video_id}
Titulo del video: {video_title}
Fecha de subida: {upload_date}
Vistas: {view_count}
Likes: {like_count}
Comentarios: {comment_count}\n''')



ID del video: A_aX6kYVOTI
Titulo del video: El horripilante hallazgo de Gary McKinnon, el hombre que HACKEÓ AL PENTÁGONO Y LO TRATARON DE MATAR
Fecha de subida: 2022-09-21
Vistas: 2384328
Likes: 176258
Comentarios: 5410

